In [1]:
import pandas as pd
import numpy as np
import librosa
import seaborn as sns
import os
import json
import IPython.display as ipd
import soundfile as sf
import math
import h5py

from glob import glob
from tqdm import tqdm
from matplotlib import pyplot as plt
from itertools import chain
from os.path import join as pjoin
from shutil import copyfile
from copy import deepcopy

from code_base.utils import write_json, load_json

/backup/vova/src/exps/bird_clef_2024/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2024

In [ ]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv", converters={"secondary_labels":eval})

In [ ]:
all_birds = list(set(train_metadata["primary_label"]) | set(chain(*train_metadata["secondary_labels"])))

In [ ]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [ ]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024.json", bird2int)

# 2024 Scored Data DF

In [ ]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_scored_meta_prev_comps_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_nodupl_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})

In [ ]:
all_birds = list(
    set(chain(*train_metadata["all_labels"])) | 
    set(chain(*train_metadata_prev_comp["all_labels"])) | 
    set(chain(*train_metadata_xeno_canto["all_labels"]))
)

In [ ]:
len(all_birds)

In [ ]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [ ]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_scoed_add_data.json", bird2int)

# 2024 + 2024 Scored from XC + Prev Comps

In [ ]:
train_metadata = pd.read_csv("/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv1.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_prev_comp = pd.read_csv("/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV2_meta_prev_comps_extended_pruned.csv", converters={"secondary_labels":eval, "all_labels": eval})
train_metadata_xeno_canto = pd.read_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_noduplV3_extended.csv", converters={"secondary_labels":eval, "all_labels": eval})

In [ ]:
all_birds = list(
    set(chain(*train_metadata["all_labels"])) | 
    set(chain(*train_metadata_prev_comp["all_labels"]))
)
len(all_birds)

In [ ]:
train_metadata_xeno_canto["all_labels"] = train_metadata_xeno_canto["all_labels"].apply(lambda x: [bird for bird in x if bird in all_birds])
train_metadata_xeno_canto["secondary_labels"] = train_metadata_xeno_canto["secondary_labels"].apply(lambda x: [bird for bird in x if bird in all_birds])

In [ ]:
train_metadata_xeno_canto.to_csv("/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_noduplV3_extended_2024PrevCompSecLabels.csv", index=False)

In [ ]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [ ]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp.json", bird2int)

# Check

In [ ]:
train_metadata = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/birdclef_2024/train_metadata_extended_noduplv2.csv",
    converters={"all_labels": eval}
)
add_data_prev_comp = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV3_scored_meta_prev_comps_extended.csv",
    converters={"all_labels": eval}
)
add_data_xc = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_2024_classes/train_metadata_noduplV4_extended_2024SecLabels.csv",
    converters={"all_labels": eval}
)
add_data_prev_comp_all = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV3_meta_prev_comps_extended.csv",
    converters={"all_labels": eval}
)

In [ ]:
add_data_prev_comp_all.shape

In [ ]:
all_birds = (
    set(chain(*train_metadata["all_labels"])) |
    set(chain(*add_data_prev_comp["all_labels"])) |
    set(chain(*add_data_xc["all_labels"])) |
    set(chain(*add_data_prev_comp_all["all_labels"]))
)

In [ ]:
json_all_birds = load_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp.json")

In [ ]:
set(json_all_birds.keys()) == all_birds

In [ ]:
add_data_prev_comp_all["primary_label"]

# 2024 + 2024 Scored from XC + Prev Comps + XC (India + VC > 50)

In [3]:
merged_train_metadata = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/birdclef_2024/merged_train_metadata_extended_noduplv2.csv",
    converters={"all_labels": eval}
)
prev_comp_data = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/dfs/full_noduplsV3_meta_prev_comps_extended.csv",
    converters={"all_labels": eval}
)
xc_add_data = pd.read_csv(
    "/home/vova/data/exps/birdclef_2024/xeno_canto/dataset_NO2024_classes/train_metadata_noduplV5_extended_india_vc50.csv",
    converters={"all_labels": eval}
)

/tmp/ipykernel_3029830/2390116080.py:1: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  merged_train_metadata = pd.read_csv(
/tmp/ipykernel_3029830/2390116080.py:5: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  prev_comp_data = pd.read_csv(


In [5]:
all_birds = (
    set(chain(*merged_train_metadata["all_labels"])) |
    set(chain(*prev_comp_data["all_labels"])) |
    set(chain(*xc_add_data["all_labels"])) 
)

len(all_birds)

6478

In [6]:
bird2int = {
    bird:idx for idx, bird in enumerate(all_birds)
}

In [8]:
write_json("/home/vova/data/exps/birdclef_2024/class_mappings/bird2int_2024_PrevComp_XCIndiaVC50.json", bird2int)